# OpenAI Function Calling (Tools) Example

This notebook demonstrates how to use function calling with OpenAI in Dinnovos Agent.

In [1]:
import os
import json
from dinnovos.llms.openai import OpenAILLM

## 1. Configure API Key

In [2]:
# Configure your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY") or "your-api-key-here"

In [3]:
# Create LLM instance
llm = OpenAILLM(api_key=api_key, model="gpt-4")

## 2. Define Tools (Functions)

In [4]:
# Define available tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Bogotá, Colombia"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g. '2 + 2'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

## 3. Implement the Functions

In [5]:
def get_weather(location: str, unit: str = "celsius") -> str:
    """Mock weather function"""
    return json.dumps({
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "condition": "Sunny",
        "humidity": 65
    })

def calculate(expression: str) -> str:
    """Calculator function"""
    try:
        result = eval(expression)
        return json.dumps({"expression": expression, "result": result})
    except Exception as e:
        return json.dumps({"error": str(e)})

# Dictionary of available functions
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

---

# 🚀 RECOMMENDED METHOD: call_with_function_execution()

**The easiest way** to use function calling. Automatically handles the entire cycle.

## Example 1: Basic Usage (Automatic)

In [6]:
# One call does it all!
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # See the complete process
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Functions called: {len(result['function_calls'])}")


📊 Context: 17 tokens (0.01% used)

Iteration 1

🔧 Calling function: get_weather
📋 Arguments: {'location': 'Bogotá, Colombia', 'unit': 'celsius'}
✅ Result: {"location": "Bogot\u00e1, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}

📊 Context: 37 tokens (0.03% used)

Iteration 2

✅ Final response: The current weather in Bogotá, Colombia is sunny with a temperature of 22 degrees Celsius and a humidity of 65%.The current weather in Bogotá, Colombia is sunny with a temperature of 22 degrees Celsius and a humidity of 65%.

📝 FINAL RESPONSE:
The current weather in Bogotá, Colombia is sunny with a temperature of 22 degrees Celsius and a humidity of 65%.The current weather in Bogotá, Colombia is sunny with a temperature of 22 degrees Celsius and a humidity of 65%.

🔄 Iterations: 2
🔧 Functions called: 1


## Example 2: Multiple Functions

In [7]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Compare the weather in Bogotá and Medellín, and calculate 25 * 4"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Total functions called: {len(result['function_calls'])}")


📊 Context: 26 tokens (0.02% used)

Iteration 1

🔧 Calling function: get_weather
📋 Arguments: {'location': 'Bogotá, Colombia', 'unit': 'celsius'}
✅ Result: {"location": "Bogot\u00e1, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}

🔧 Calling function: get_weather
📋 Arguments: {'location': 'Medellín, Colombia', 'unit': 'celsius'}
✅ Result: {"location": "Medell\u00edn, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}

🔧 Calling function: calculate
📋 Arguments: {'expression': '25 * 4'}
✅ Result: {"expression": "25 * 4", "result": 100}

📊 Context: 86 tokens (0.07% used)

Iteration 2

✅ Final response: The weather in both Bogotá and Medellín, Colombia is currently sunny with a temperature of 22 degrees Celsius and a humidity of 65%. 

The result of the calculation 25 * 4 is 100.The weather in both Bogotá and Medellín, Colombia is currently sunny with a temperature of 22 degrees Celsius and a humidity of 65%. 

The 

## Example 3: Silent Mode

In [8]:
# Without verbose - just get the result
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Calculate (100 + 50) / 3"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False
)

print("Response:", result['content'])

Response: The result of the calculation (100 + 50) / 3 is 50.The result of the calculation (100 + 50) / 3 is 50.


## Example 4: View Complete History

In [9]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the temperature in Cali?"}],
    tools=tools,
    available_functions=available_functions
)

print("\n📊 COMPLETE ANALYSIS:")
print(f"\n✅ Response: {result['content']}")
print(f"\n🔄 Iterations performed: {result['iterations']}")
print(f"\n🔧 Functions executed:")
for i, func_call in enumerate(result['function_calls'], 1):
    print(f"\n  {i}. {func_call['name']}")
    print(f"     Args: {func_call['arguments']}")
    print(f"     Result: {func_call['result']}")


📊 COMPLETE ANALYSIS:

✅ Response: The current temperature in Cali, Colombia is 22°C. The weather is sunny with a humidity of 65%.The current temperature in Cali, Colombia is 22°C. The weather is sunny with a humidity of 65%.

🔄 Iterations performed: 2

🔧 Functions executed:

  1. get_weather
     Args: {'location': 'Cali, Colombia', 'unit': 'celsius'}
     Result: {"location": "Cali, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}


## Example 5: Streaming with Function Execution

In [10]:
print("=" * 70)
print("STREAMING WITH AUTOMATIC FUNCTION EXECUTION")
print("=" * 70)

for chunk in llm.call_stream_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # This will show debug info
):
    chunk_type = chunk.get("type")
    
    if chunk_type == "iteration_start":
        print(f"\n[Iteration {chunk.get('iteration')}]")
    
    elif chunk_type == "text_delta":
        # Stream text as it arrives
        print(chunk.get("content"), end="", flush=True)
    
    elif chunk_type == "function_call_start":
        # Function is being called
        print(f"\n🔧 Calling: {chunk.get('function_name')}")
        print(f"   Args: {chunk.get('arguments')}")
    
    elif chunk_type == "function_call_result":
        # Function completed
        print(f"✅ Result: {chunk.get('result')}")
    
    elif chunk_type == "final":
        # Final response
        print(f"\n\n{'='*70}")
        print(f"Completed in {chunk.get('iterations')} iterations")
        print(f"Functions called: {len(chunk.get('function_calls', []))}")
    
    elif chunk_type == "error":
        print(f"\n❌ Error: {chunk.get('content')}")

STREAMING WITH AUTOMATIC FUNCTION EXECUTION

📊 Context: 17 tokens (0.01% used)

Iteration 1

[Iteration 1]
[DEBUG] Event type: response.created
[DEBUG] Event type: response.in_progress
[DEBUG] Event type: response.output_item.added
[DEBUG] Function call started: get_weather
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event type: response.function_call_arguments.delta
[DEBUG] Event typ

## Example 6: Streaming with Function Execution silently

In [11]:
for chunk in llm.call_stream_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False  # This will show debug info
):
    chunk_type = chunk.get("type")
    
    if chunk_type == "iteration_start":
        print(f"\n[Iteration {chunk.get('iteration')}]")
    
    elif chunk_type == "text_delta":
        # Stream text as it arrives
        print(chunk.get("content"), end="", flush=True)
    
    elif chunk_type == "function_call_start":
        # Function is being called
        print(f"\n🔧 Calling: {chunk.get('function_name')}")
        print(f"   Args: {chunk.get('arguments')}")
    
    elif chunk_type == "function_call_result":
        # Function completed
        print(f"✅ Result: {chunk.get('result')}")
    
    elif chunk_type == "final":
        # Final response
        print(f"\n\n{'='*70}")
        print(f"Completed in {chunk.get('iterations')} iterations")
        print(f"Functions called: {len(chunk.get('function_calls', []))}")
    
    elif chunk_type == "error":
        print(f"\n❌ Error: {chunk.get('content')}")


[Iteration 1]

🔧 Calling: get_weather
   Args: {'location': 'Bogotá, Colombia', 'unit': 'celsius'}
✅ Result: {"location": "Bogot\u00e1, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}

[Iteration 2]
The weather in Bogotá, Colombia is currently sunny with a temperature of 22 degrees Celsius and a humidity of 65%.

Completed in 2 iterations
Functions called: 1


## Example 6: Streaming with Multiple Functions

Test streaming with a more complex query that requires multiple function calls.

In [12]:
print("=" * 70)
print("STREAMING: Multiple Functions")
print("=" * 70)

for chunk in llm.call_stream_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá and calculate 15 * 7"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False  # Set to True to see debug info
):
    chunk_type = chunk.get("type")
    
    if chunk_type == "iteration_start":
        iteration = chunk.get('iteration')
        if iteration > 1:
            print(f"\n\n[Iteration {iteration}]")
    
    elif chunk_type == "text_delta":
        print(chunk.get("content"), end="", flush=True)
    
    elif chunk_type == "function_call_start":
        print(f"\n\n🔧 Calling: {chunk.get('function_name')}")
        print(f"   Args: {chunk.get('arguments')}")
    
    elif chunk_type == "function_call_result":
        print(f"✅ Result: {chunk.get('result')}")
    
    elif chunk_type == "final":
        print(f"\n\n{'='*70}")
        print(f"✅ Completed in {chunk.get('iterations')} iterations")
        
        for i, func_call in enumerate(chunk.get('function_calls', []), 1):
            print(f"\n{i}. {func_call['name']}: {func_call['result']}")
    
    elif chunk_type == "error":
        print(f"\n❌ Error: {chunk.get('content')}")

STREAMING: Multiple Functions


🔧 Calling: get_weather
   Args: {'location': 'Bogotá, Colombia', 'unit': 'celsius'}
✅ Result: {"location": "Bogot\u00e1, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}


🔧 Calling: calculate
   Args: {'expression': '15 * 7'}
✅ Result: {"expression": "15 * 7", "result": 105}


[Iteration 2]
The current weather in Bogotá, Colombia is 22 degrees Celsius, with sunny conditions and a humidity of 65%. The result of 15 * 7 is 105.

✅ Completed in 2 iterations

1. get_weather: {"location": "Bogot\u00e1, Colombia", "temperature": 22, "unit": "celsius", "condition": "Sunny", "humidity": 65}

2. calculate: {"expression": "15 * 7", "result": 105}
